# 0. Import Dependencies

In [1]:
import os
import cv2
import numpy as np

import pandas as pd

import joblib

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split


# 1. Prepare Samples

In [ ]:
positive_samples_paths = []
negative_samples_paths = []

positive_samples_df = pd.read_csv("../../datasets/faces/faces.csv")
# # Remove images with more than one face
# positive_samples_df = positive_samples_df.groupby('image_name').filter(lambda x: len(x) == 1).reset_index(drop=True)
# # Shuffle the data frame
# positive_samples_df = positive_samples_df.sample(frac=1).reset_index(drop=True)

negative_samples_dirs = [os.path.join("../../datasets/faces/natural_images/", folder) for folder in os.listdir("../../datasets/faces/natural_images/") if folder != "person"]

for dir in negative_samples_dirs: 
    for image in os.listdir(dir):
        negative_samples_paths.append(os.path.join(dir, image))

for image in os.listdir("../../datasets/faces/natural_images/person/"):
    positive_samples_paths.append(os.path.join("../../datasets/faces/natural_images/person/", image))
# for image in os.listdir("../../datasets/faces/images"):
#     positive_samples_paths.append(os.path.join(dir, image))


# 2. Feature Extraction

In [2]:
feature_vector = []
label_vector = []

FEATURE_WINDOW_SIZE = (64, 64)

win_size = FEATURE_WINDOW_SIZE 
block_size = (16, 16)
block_stride = (8, 8)
cell_size = (8, 8)
num_bins = 9

HOG = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)

def extract_features(Image):

    gray_image = cv2.cvtColor(Image, cv2.COLOR_BGR2GRAY)

    equalized_image = cv2.equalizeHist(gray_image)

    resized_image = cv2.resize(equalized_image, FEATURE_WINDOW_SIZE)
        
    feature = HOG.compute(resized_image).flatten()

    if feature.max() > 0:
        feature /= feature.max()

    return feature

In [ ]:
for index, image_properties in positive_samples_df.iterrows():

    image_path = os.path.join("../../datasets/faces/images", image_properties["image_name"])
    image = cv2.imread(image_path)

    x0 = int(image_properties["x0"])
    x1 = int(image_properties["x1"])
    y0 = int(image_properties["y0"])
    y1 = int(image_properties["y1"])

    face = image[y0:y1 ,x0:x1]
    feature_vector.append( extract_features(face) )
    label_vector.append( 1 )


for image_path in positive_samples_paths:

    face = cv2.imread(image_path)
    feature_vector.append( extract_features(face) )
    label_vector.append( 1 )


for image_path in negative_samples_paths:

    not_face = cv2.imread(image_path)
    feature_vector.append( extract_features(not_face) )
    label_vector.append( 0 )


# 3. Train and Save the model

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(feature_vector, label_vector, test_size=0.05, random_state=42)

face_classifer = LinearSVC().fit(x_train, y_train)

score = face_classifer.score(x_test, y_test)

print("Accuracy: {:.2f}".format(score))

joblib.dump(face_classifer, "FaceClassifier_HoG{}{}{}({:.2f}).pkl".format(win_size, block_size, cell_size ,score))

In [3]:
def union_boxes_min_max(boxes, min_x, min_y, max_x, max_y):
  """
  Unifies a list of bounding boxes using minimum and maximum values.

  Args:
    boxes: A list of bounding boxes represented as (x_min, y_min, x_max, y_max) tuples.

  Returns:
    unified_box: A tuple representing the unified bounding box (x_min, y_min, x_max, y_max).
  """

  overall_box = None

  if not overall_box:
    for box in boxes:
      x_min, y_min, x_max, y_max = box
      min_x = min(min_x, x_min)
      min_y = min(min_y, y_min)
      max_x = max(max_x, x_max)
      max_y = max(max_y, y_max)
    overall_box = (min_x, min_y, max_x, max_y)
    
    return overall_box

def filter_boxes_with_skin_mask(boxes, skin_mask):
  """
  Filters bounding boxes based on overlap with a skin mask.

  Args:
    boxes: A list of bounding boxes (x_min, y_min, x_max, y_max)
    skin_mask: A binary mask (0 or 255) where 255 represents skin pixels.

  Returns:
    filtered_boxes: A list of remaining bounding boxes after filtering.
  """
  filtered_boxes = []
  for box in boxes:
    x_min, y_min, x_max, y_max = box
    # Calculate area of the bounding box
    box_area = (x_max - x_min) * (y_max - y_min)
    # Get the portion of the bounding box within the skin mask
    intersection_mask = skin_mask[y_min:y_max, x_min:x_max]
    intersection_area = cv2.countNonZero(intersection_mask)
    # Calculate intersection-over-union (IoU) ratio
    iou = intersection_area / box_area

    # Define threshold for acceptable overlap with skin mask
    threshold = 0.5

    if iou >= threshold:
      filtered_boxes.append(box)

  return filtered_boxes  

def get_bounding_box(mask):
    """Extracts the bounding box coordinates of a segmented region in a binary mask.

    Args:
        mask: A 2D binary mask (0s for background, 1s for foreground).

    Returns:
        A tuple containing the top-left, bottom-right coordinates of the bounding box: (x0, y0, x1, y1).
    """

    # Find non-zero pixels (segmented region)
    non_zero_pixels = np.nonzero(mask)

    # Extract the minimum and maximum row and column indices
    min_row, min_col = np.min(non_zero_pixels, axis=1)
    max_row, max_col = np.max(non_zero_pixels, axis=1)

    # Return coordinates in desired format (x0, y0, x1, y1)
    return min_col, min_row, max_col, max_row  # Note the order of coordinates

def segment_skin(image):
  """
  Segments skin pixels based on chrominance analysis.

  Args:
    image: A BGR image.

  Returns:
    skin_mask: A binary mask (0 or 255) where 255 represents skin pixels.
  """
  # Convert to YCbCr color space
  ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

  # Define chrominance thresholds for skin
  lower_thresh = (0, 133, 77)
  upper_thresh = (255, 173, 127)

  # Create binary mask based on thresholds
  skin_mask = cv2.inRange(ycrcb, lower_thresh, upper_thresh)

  # Apply morphological operations for smoothing
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
  skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_OPEN, kernel, iterations=3)

  return skin_mask


# 4. Real-Time Testing

In [5]:
face_classifer = joblib.load("FaceClassifier_HoG(64, 64)(16, 16)(8, 8)(0.95).pkl")

camera = cv2.VideoCapture(0)

window_sizes = [(64, 64), (96, 96)]

while camera.isOpened():

    detected_faces = []

    _, frame = camera.read()

    skin_mask = segment_skin(frame)

    x0, y0, x1, y1 = get_bounding_box(skin_mask)

    for window_size in window_sizes:

        y_stride = window_size[0]
        x_stride = window_size[1]

        for y in range(y0, y1, y_stride):
        # for y in range(0, frame.shape[0] - window_size[0], y_stride):
            
            for x in range(x0, x1, x_stride):
            # for x in range(0, frame.shape[1] - window_size[1], x_stride):
                # Extract window and predict
                window = frame[y : y+window_size[0], x : x+window_size[1]]

                [prediction] = face_classifer.predict([extract_features(window)])

                # Check prediction and register face
                if prediction == 1:  # Adjust threshold based on your model
                    detected_faces.append([x, y, x + window_size[1], y + window_size[0]])

                    # cv2.rectangle(frame, (x, y), (x + window_size[1], y + window_size[0]), (0, 255, 0), 2)


    detected_faces = filter_boxes_with_skin_mask(detected_faces, skin_mask)

    detected_faces = union_boxes_min_max(detected_faces, x0, y0, x1, y1)

    cv2.rectangle(frame, (detected_faces[0], detected_faces[1]), (detected_faces[2], detected_faces[3]), (0, 255, 0), 2)

    cv2.imshow("BGR Frame", frame)
    cv2.imshow('Skin Mask', skin_mask)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close the OpenCV windows
camera.release()
cv2.destroyAllWindows()
